In [2]:
!pip install sagemaker ipywidgets --upgrade --quiet


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
model_id, model_version = "tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4", "*"


In [8]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
training_instance_type = "ml.g4dn.xlarge"
# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

Using model 'tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4' with wildcard version identifier '*'. You can pin to version '4.0.2' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


In [11]:
# Sample training data is available in this bucket
training_data_bucket = "sagemaker-us-east-1-529165531209"
training_data_prefix = "isic_image/train"

training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

output_bucket = sess.default_bucket()
output_prefix = "jumpstart-example-ic-training"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

In [14]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
hyperparameters["epochs"] = "5"
hyperparameters['optimizer'] = 'sgd'
print(str(hyperparameters).replace(", ", "\n"))

{'train_only_top_layer': 'True'
'epochs': '5'
'batch_size': '32'
'optimizer': 'sgd'
'learning_rate': '0.001'
'beta_1': '0.9'
'beta_2': '0.999'
'momentum': '0.9'
'epsilon': '1e-07'
'rho': '0.95'
'initial_accumulator_value': '0.1'
'reinitialize_top_layer': 'Auto'
'early_stopping': 'False'
'early_stopping_patience': '5'
'early_stopping_min_delta': '0.0'
'dropout_rate': '0.2'
'regularizers_l2': '0.0001'
'label_smoothing': '0.1'
'image_resize_interpolation': 'bilinear'
'augmentation': 'False'
'augmentation_random_flip': 'horizontal_and_vertical'
'augmentation_random_rotation': '0.2'
'augmentation_random_zoom': '0.1'
'binary_mode': 'False'
'eval_metric': 'accuracy'
'validation_split_ratio': '0.2'
'random_seed': '123'}


In [15]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")

training_metric_definitions = [
    {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
    {"Name": "train_accuracy", "Regex": "- accuracy: ([0-9\\.]+)"},
    {"Name": "train_loss", "Regex": "- loss: ([0-9\\.]+)"},
]

# Create SageMaker Estimator instance
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
    metric_definitions=training_metric_definitions,
)

In [17]:
ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

INFO:sagemaker:Creating training-job with name: jumpstart-example-tensorflow-ic-imagene-2024-03-18-20-55-35-381


2024-03-18 20:55:35 Starting - Starting the training job...
2024-03-18 20:55:50 Starting - Preparing the instances for training...
2024-03-18 20:56:23 Downloading - Downloading input data...
2024-03-18 20:56:53 Downloading - Downloading the training image.....................
2024-03-18 21:00:29 Training - Training image download completed. Training in progress..2024-03-18 21:00:38.815617: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-03-18 21:00:38.816120: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-03-18 21:00:38.816348: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0

In [18]:
training_job_name = ic_estimator.latest_training_job.job_name

In [19]:
import sagemaker
from IPython.core.display import Markdown

sagemaker_session = sagemaker.Session()

link = (
    "https://console.aws.amazon.com/cloudwatch/home?region="
    + sagemaker_session.boto_region_name
    + "#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20"
    + training_job_name
)
display(Markdown("CloudWatch metrics: [link](" + link + ")"))

CloudWatch metrics: [link](https://console.aws.amazon.com/cloudwatch/home?region=us-east-1#metricsV2:query=%7B/aws/sagemaker/TrainingJobs,TrainingJobName%7D%20jumpstart-example-tensorflow-ic-imagene-2024-03-18-20-55-35-381)

In [21]:
from sagemaker import TrainingJobAnalytics

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()

df

,timestamp,metric_name,value
0,0.0,val_accuracy,0.422455
1,0.0,val_loss,1.677205
2,0.0,train_accuracy,0.331905
3,0.0,train_loss,1.921056


In [23]:
inference_instance_type = "ml.p2.xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = ic_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-529165531209/jumpstart-example-ic-training/output/jumpstart-example-tensorflow-ic-imagene-2024-03-18-20-55-35-381/output/model.tar.gz), script artifact (s3://jumpstart-cache-prod-us-east-1/source-directory-tarballs/tensorflow/inference/ic/v2.0.3/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-529165531209/sagemaker-jumpstart-2024-03-18-21-11-00-957/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: sagemaker-jumpstart-2024-03-18-21-11-00-957
INFO:sagemaker:Creating endpoint-config with name jumpstart-example-FT-tensorflow-ic-imag-2024-03-18-21-11-00-957
INFO:sagemaker:Creating endpoint with name jumpstart-example-FT-tensorflow-ic-imag-2024-03-18-21-11-00-957


--------------!

In [24]:
s3_bucket = "sagemaker-us-east-1-529165531209"
key_prefix = "isic_image/test"


def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)


skin_images = {
    "img1.jpg": "dermatofibroma/ISIC_0025154.jpg",
    "img2.jpg": "nevus/ISIC_0014687_downsampled.jpg",
}
download_from_s3(skin_images)

In [25]:
from IPython.core.display import HTML

for image_filename in skin_images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = finetuned_predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    model_predictions = json.loads(query_response)
    predicted_label = model_predictions["predicted_label"]
    display(
        HTML(
            f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
            f"<figcaption>Predicted Label: {predicted_label}</figcaption>"
        )
    )

In [26]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: sagemaker-jumpstart-2024-03-18-21-11-00-957
INFO:sagemaker:Deleting endpoint configuration with name: jumpstart-example-FT-tensorflow-ic-imag-2024-03-18-21-11-00-957
INFO:sagemaker:Deleting endpoint with name: jumpstart-example-FT-tensorflow-ic-imag-2024-03-18-21-11-00-957
